In [1]:
from tensorflow import keras
keras.__version__

'2.4.0'

# A first look at a neural network

This notebook contains the code samples found in Chapter 2, Section 1 of [Deep Learning with Python](https://www.manning.com/books/deep-learning-with-python?a_aid=keras&a_bid=76564dff). Note that the original text features far more content, in particular further explanations and figures: in this notebook, you will only find source code and related comments.

----

We will now take a look at a first concrete example of a neural network, which makes use of Keras and [Orca](https://analytics-zoo.github.io/master/#Orca/overview/) to learn to classify 
hand-written digits. Unless you already have experience with Keras or similar libraries, you will not understand everything about this 
first example right away. You probably haven't even installed Keras yet. Don't worry, that is perfectly fine. In the next chapter, we will 
review each element in our example and explain them in detail. So don't worry if some steps seem arbitrary or look like magic to you! 
We've got to start somewhere.

The problem we are trying to solve here is to classify grayscale images of handwritten digits (28 pixels by 28 pixels), into their 10 
categories (0 to 9). The dataset we will use is the MNIST dataset, a classic dataset in the machine learning community, which has been 
around for almost as long as the field itself and has been very intensively studied. It's a set of 60,000 training images, plus 10,000 test 
images, assembled by the National Institute of Standards and Technology (the NIST in MNIST) in the 1980s. You can think of "solving" MNIST 
as the "Hello World" of deep learning -- it's what you do to verify that your algorithms are working as expected. As you become a machine 
learning practitioner, you will see MNIST come up over and over again, in scientific papers, blog posts, and so on.

The MNIST dataset comes pre-loaded in Keras, in the form of a set of four Numpy arrays:

In [2]:
from tensorflow.keras.datasets import mnist

(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

`train_images` and `train_labels` form the "training set", the data that the model will learn from. The model will then be tested on the 
"test set", `test_images` and `test_labels`. Our images are encoded as Numpy arrays, and the labels are simply an array of digits, ranging 
from 0 to 9. There is a one-to-one correspondence between the images and the labels.

Let's have a look at the training data:

In [3]:
train_images.shape

(60000, 28, 28)

In [4]:
train_data_size = len(train_labels)
train_data_size

60000

In [5]:
train_labels

array([5, 0, 4, ..., 5, 6, 8], dtype=uint8)

Let's have a look at the test data:

In [6]:
test_images.shape

(10000, 28, 28)

In [7]:
test_data_size = len(test_labels)
test_data_size

10000

In [8]:
test_labels

array([7, 2, 1, ..., 4, 5, 6], dtype=uint8)

Our workflow will be as follow: first we will present our neural network with the training data, `train_images` and `train_labels`. The 
network will then learn to associate images and labels. Finally, we will ask the network to produce predictions for `test_images`, and we 
will verify if these predictions match the labels from `test_labels`.

Let's build our network -- again, remember that you aren't supposed to understand everything about this example just yet.

In [9]:
def model_creator(config):
    network = keras.Sequential(
        [keras.layers.Dense(512, activation='relu', input_shape=(28 * 28, )),
         keras.layers.Dense(10, activation='softmax'),
         ]
    )

    network.compile(optimizer='rmsprop',
                    loss='sparse_categorical_crossentropy',
                    metrics=['accuracy'])
    return network


The core building block of neural networks is the "layer", a data-processing module which you can conceive as a "filter" for data. Some 
data comes in, and comes out in a more useful form. Precisely, layers extract _representations_ out of the data fed into them -- hopefully 
representations that are more meaningful for the problem at hand. Most of deep learning really consists of chaining together simple layers 
which will implement a form of progressive "data distillation". A deep learning model is like a sieve for data processing, made of a 
succession of increasingly refined data filters -- the "layers".

Here our network consists of a sequence of two `Dense` layers, which are densely-connected (also called "fully-connected") neural layers. 
The second (and last) layer is a 10-way "softmax" layer, which means it will return an array of 10 probability scores (summing to 1). Each 
score will be the probability that the current digit image belongs to one of our 10 digit classes.

To make our network ready for training, we need to pick three more things, as part of "compilation" step:

* A loss function: the is how the network will be able to measure how good a job it is doing on its training data, and thus how it will be 
able to steer itself in the right direction.
* An optimizer: this is the mechanism through which the network will update itself based on the data it sees and its loss function.
* Metrics to monitor during training and testing. Here we will only care about accuracy (the fraction of the images that were correctly 
classified).

Then we need to define two configuration parameters, which are:

* `max_epoch`: the maximum number of passes of the entire training dataset.
*  `batch_size`: the number of training examples utilized in one iteration.

As an example, we can define them as follows:

In [10]:
max_epoch = 5
batch_size = 320

Before training, we will preprocess our data by reshaping it into the shape that the network expects, and scaling it so that all values are in the `[0, 1]` interval. Previously, our training images for instance were stored in an array of shape `(60000, 28, 28)` of type uint8 with values in the `[0, 255]` interval (`train_data_size` = 60000). We transform it into a `float32` array of shape `(60000, 28 * 28)` with values between 0 and 1. The datasets to be used are then created using the slices of the original trainging and testing data arrays. In addition, the "repeat" and "shuffle" process will help the model avoid bias and converge faster.

In [11]:
import tensorflow as tf

def train_data_creator(config, batch_size):
    (train_images, train_labels), _ = mnist.load_data()
    train_images = train_images.reshape((train_data_size, 28 * 28))
    train_images = train_images.astype('float32') / 255

    dataset = tf.data.Dataset.from_tensor_slices((train_images, train_labels))
    dataset = dataset.repeat()
    dataset = dataset.shuffle(1000)
    dataset = dataset.batch(batch_size)
    return dataset

def val_data_creator(config, batch_size):
    _, (test_images, test_labels) = mnist.load_data()
    test_images = test_images.reshape((test_data_size, 28 * 28))
    test_images = test_images.astype('float32') / 255

    dataset = tf.data.Dataset.from_tensor_slices((test_images, test_labels))
    dataset = dataset.repeat()
    dataset = dataset.batch(batch_size)
    return dataset

We are now ready to train our network using Orca Estimator. Firstly, we need to initialize the Orca context by specifying local or distributed mode.

In [12]:
from zoo.orca import init_orca_context, stop_orca_context
from zoo.orca import OrcaContext

OrcaContext.log_output = True # (this will display terminal's stdout and stderr in the Jupyter notebook).
cluster_mode = "local"

if cluster_mode == "local":
    init_orca_context(cluster_mode="local", cores=4)  # run in local mode
elif cluster_mode == "k8s":
    init_orca_context(cluster_mode="k8s", num_nodes=2, cores=2)  # run on K8s cluster
elif cluster_mode == "yarn":
    init_orca_context(cluster_mode="yarn-client", num_nodes=2, cores=2)  # run on Hadoop YARN cluster

After importing the network configurations from Keras, we can call the `fit` method to fit the model to its training data. The number of training steps (or validation steps) per epoch can be simply calculated by `train_data_size // batch_size` (or `test_data_size // batch_size`). When training finishes, the model can be saved using the `save` method.

In [13]:
from zoo.orca.learn.tf2 import Estimator

est = Estimator.from_keras(model_creator=model_creator, workers_per_node=2)
est.fit(data=train_data_creator,
        epochs=max_epoch,
        batch_size=batch_size,
        steps_per_epoch=train_data_size // batch_size,
        validation_data=val_data_creator,
        validation_steps=test_data_size // batch_size)
est.save("/tmp/mnist_keras.ckpt")

...
(pid=3348) Epoch 5/5
187/187 [==============================] - 1s 6ms/step - loss: 0.0522 - accuracy: 0.9850 - val_loss: 0.0743 - val_accuracy: 0.9767


'/tmp/mnist_keras.ckpt'

Two quantities are being displayed during training: the "loss" of the network over the training data, and the accuracy of the network over 
the training data.

We quickly reach an accuracy of 0.9850 (i.e. 98.50%) on the training data. Now let's check that our model performs well on the test set too:

In [14]:
stats = est.evaluate(val_data_creator, num_steps=test_data_size // batch_size)
est.shutdown()
print(stats)

31/31 [==============================] - 0s 978us/step - loss: 0.0707 - accuracy: 0.9788
{'validation_loss': 0.07073836028575897, 'validation_accuracy': 0.9788306355476379}



Our test set accuracy turns out to be 97.88% -- that's a bit lower than the training set accuracy. 
This gap between training accuracy and test accuracy is an example of "overfitting", 
the fact that machine learning models tend to perform worse on new data than on their training data. 

This concludes our very first example -- you just saw how we could build and a train a neural network to classify handwritten digits. In the next chapter, we will go in detail over every moving piece we just previewed, and clarify what is really 
going on behind the scenes. You will learn about "tensors", the data-storing objects going into the network, about tensor operations, which 
layers are made of, and about gradient descent, which allows our network to learn from its training examples.

Note: you should call `stop_orca_context()` when the program finishes.

In [15]:
stop_orca_context()

Stopping orca context
